## IMPORTS

In [1]:
import os
import cv2

import numpy as np
from numpy.random import seed
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import regex as re

from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.random import set_seed

# 1-Off Accuracy

## Loading Inputs

In [15]:
inputs_path = "../data/cnn_inputs/"

In [16]:
y_test_cat = np.load("../data/cnn_inputs/y_test_cat.npy")
y_test = np.load("../data/cnn_inputs/y_test.npy")
y_pred = np.load("../data/cnn_inputs/y_pred.npy")

In [19]:
print(len(y_pred),"....",type(y_pred))
print(len(y_test_cat),"....",type(y_test_cat))
print(len(y_test),"....",type(y_test))

4402 .... <class 'numpy.ndarray'>
4402 .... <class 'numpy.ndarray'>
4402 .... <class 'numpy.ndarray'>


In [62]:
def one_off_accuracy(y_test,y_pred):
    
    try:
        i_ind = []
        accurate = 0
    
        hit_miss = pd.DataFrame(data = 0, dtype="int16"
                              ,index = np.linspace(0,y_test.shape[1]-1,y_test.shape[1],dtype="int8")
                              ,columns = ['Hit','Miss'])

        conf = pd.DataFrame(data = 0, dtype="int16"
                              ,index = np.linspace(0,y_test.shape[1]-1,y_test.shape[1],dtype="int8")
                              ,columns = np.linspace(0,y_test.shape[1]-1,y_test.shape[1],dtype="int8"))

        for i in range(len(y_pred)):

            # 1-Off Accuracy
            i_ind = [y_pred[i].argmax()-1,y_pred[i].argmax(),y_pred[i].argmax()+1]
            if y_test[i].argmax() in i_ind:
                accurate += 1

            # Hit/Miss Matrix
                hit_miss.at[y_test[i].argmax(),"Hit"] += 1
            else:
                hit_miss.at[y_test[i].argmax(),"Miss"] += 1

            # Confusion Matrix
            conf.at[y_test[i].argmax(),y_pred[i].argmax()] += 1

        conf["Total"] = np.sum(y_test,axis=0,dtype="int16")
        for k in range(0,y_test.shape[1]):
            if (conf.iloc[k,k] !=0) and (conf.at[k,"Total"]!=0):
                conf.at[k,"Hit Rate"] = round(100*np.divide(conf.at[k,k],conf.at[k,"Total"]),1)
        print("One")
        print(f"The 1-Off Accuracy is {round(100*accurate/len(y_test),2)}%")
        
    except:
        i_ind = []
        accurate = 0
        hit_miss = pd.DataFrame(data = 0, dtype="int16"
                              ,index = np.linspace(0,len(np.unique(y_test))-1,len(np.unique(y_test)),dtype="int8")
                              ,columns = ['Hit','Miss'])

        conf = pd.DataFrame(data = 0, dtype="int16"
                              ,index = np.linspace(0,len(np.unique(y_test))-1,len(np.unique(y_test)),dtype="int8")
                              ,columns = np.linspace(0,len(np.unique(y_test))-1,len(np.unique(y_test)),dtype="int8"))

        for i in range(len(y_pred)):

            # 1-Off Accuracy
            i_ind = [y_pred[i].argmax()-1,y_pred[i].argmax(),y_pred[i].argmax()+1]
            if y_test[i] in i_ind:
                accurate += 1

            # Hit/Miss Matrix
                hit_miss.at[y_test[i],"Hit"] += 1
            else:
                hit_miss.at[y_test[i],"Miss"] += 1

            # Confusion Matrix
            conf.at[y_test[i],y_pred[i].argmax()] += 1

        conf["Total"] = np.sum(y_test_cat,axis=0,dtype="int16")
        for k in range(0,len(np.unique(y_test))):
            if (conf.iloc[k,k] !=0) and (conf.at[k,"Total"]!=0):
                conf.at[k,"Hit Rate"] = round(100*np.divide(conf.at[k,k],conf.at[k,"Total"]),1)
        print("Two")
        print(f"The 1-Off Accuracy is {round(100*accurate/len(y_test),2)}%")
    return hit_miss,conf

In [64]:
hit_miss,conf = one_off_accuracy(y_test,y_pred)

Two
The 1-Off Accuracy is 70.2%


In [55]:
conf

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,Total,Hit Rate
0,357,44,4,3,0,8,0,1,0,0,0,0,0,1,0,0,418,85.4
1,28,90,19,15,2,18,0,1,0,0,0,1,0,0,0,0,174,51.7
2,2,48,24,25,7,28,0,1,0,0,2,0,0,0,0,0,137,17.5
3,0,6,5,28,23,136,0,2,0,0,1,0,0,1,0,0,202,13.9
4,0,2,2,8,44,462,0,8,0,0,2,1,0,0,0,0,529,8.3
5,0,1,3,6,33,823,0,65,0,0,12,1,0,0,0,0,944,87.2
6,0,1,2,2,2,322,1,86,0,1,27,2,0,0,0,0,446,0.2
7,0,0,1,0,0,188,0,110,0,1,49,5,0,0,0,0,354,31.1
8,0,0,0,1,0,78,0,73,0,2,49,7,0,0,0,0,210,NaN
9,0,1,0,0,0,54,0,61,0,0,69,17,0,0,0,0,202,NaN


In [10]:
hit_miss

,Hit,Miss
0,920,328
1,315,123
2,74,47
3,77,91
4,30,157
5,423,63
6,791,253
7,306,239
8,0,0
9,0,0
